In [567]:
import pandas as pd
import numpy as np
import pickle
import json
import ast
from surprise import Dataset
from surprise import Reader
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [505]:
game_data = pd.read_csv('../data/steamspy_data.csv')

In [506]:
game_data.head(50)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,185686,4807,0,"10,000,000 .. 20,000,000",9363,426,262,323,199,999,80,"English, French, German, Italian, Spanish - Sp...",Action,11955,"{'Action': 5372, 'FPS': 4796, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,5235,874,0,"2,000,000 .. 5,000,000",852,3,27,3,99,499,80,"English, French, German, Italian, Spanish - Sp...",Action,94,"{'Action': 745, 'FPS': 306, 'Multiplayer': 258..."
2,30,Day of Defeat,Valve,Valve,NaN,4885,541,0,"5,000,000 .. 10,000,000",811,0,16,0,99,499,80,"English, French, German, Italian, Spanish - Spain",Action,119,"{'FPS': 785, 'World War II': 246, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1791,403,0,"5,000,000 .. 10,000,000",271,0,12,0,99,499,80,"English, French, German, Italian, Spanish - Sp...",Action,10,"{'Action': 628, 'FPS': 138, 'Classic': 106, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,12501,638,0,"5,000,000 .. 10,000,000",1919,3,171,5,99,499,80,"English, French, German, Korean",Action,122,"{'FPS': 879, 'Action': 321, 'Classic': 250, 'S..."
5,60,Ricochet,Valve,Valve,NaN,3583,823,0,"5,000,000 .. 10,000,000",228,0,3,0,99,499,80,"English, French, German, Italian, Spanish - Sp...",Action,8,"{'Action': 585, 'FPS': 128, 'Multiplayer': 103..."
6,70,Half-Life,Valve,Valve,NaN,65013,2344,0,"5,000,000 .. 10,000,000",1241,58,156,60,199,999,80,"English, French, German, Italian, Spanish - Sp...",Action,894,"{'FPS': 2214, 'Sci-fi': 1749, 'Action': 1746, ..."
7,80,Counter-Strike: Condition Zero,Valve,Valve,NaN,18108,1771,0,"5,000,000 .. 10,000,000",1420,1,32,1,199,999,80,"English, French, German, Italian, Spanish - Sp...",Action,488,"{'Action': 1356, 'FPS': 1010, 'Shooter': 745, ..."
8,130,Half-Life: Blue Shift,Gearbox Software,Valve,NaN,8929,868,0,"10,000,000 .. 20,000,000",1920,71,121,71,99,499,80,"English, French, German",Action,55,"{'FPS': 454, 'Action': 281, 'Sci-fi': 211, 'Si..."
9,220,Half-Life 2,Valve,Valve,NaN,128896,3490,0,"10,000,000 .. 20,000,000",989,387,380,387,199,999,80,"English, French, German, Italian, Korean, Span...",Action,1041,"{'FPS': 3844, 'Action': 2731, 'Sci-fi': 2378, ..."


In [507]:
game_data = game_data[['appid', 'name', 'genre', 'tags']]

In [508]:
game_data.head()

,appid,name,genre,tags
0,10,Counter-Strike,Action,"{'Action': 5372, 'FPS': 4796, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Action,"{'Action': 745, 'FPS': 306, 'Multiplayer': 258..."
2,30,Day of Defeat,Action,"{'FPS': 785, 'World War II': 246, 'Multiplayer..."
3,40,Deathmatch Classic,Action,"{'Action': 628, 'FPS': 138, 'Classic': 106, 'M..."
4,50,Half-Life: Opposing Force,Action,"{'FPS': 879, 'Action': 321, 'Classic': 250, 'S..."


In [509]:
game_data.set_index('appid', inplace=True)

In [510]:
game_data

,name,genre,tags
appid,,,
10,Counter-Strike,Action,"{'Action': 5372, 'FPS': 4796, 'Multiplayer': 3..."
20,Team Fortress Classic,Action,"{'Action': 745, 'FPS': 306, 'Multiplayer': 258..."
30,Day of Defeat,Action,"{'FPS': 785, 'World War II': 246, 'Multiplayer..."
40,Deathmatch Classic,Action,"{'Action': 628, 'FPS': 138, 'Classic': 106, 'M..."
50,Half-Life: Opposing Force,Action,"{'FPS': 879, 'Action': 321, 'Classic': 250, 'S..."
...,...,...,...
1483870,Draw & Guess,"Casual, Indie","{'Casual': 264, 'Multiplayer': 245, 'Hand-draw..."
1517290,Battlefield 2042,"Action, Adventure, Casual","{'Shooter': 445, 'Action': 407, 'Multiplayer':..."
1520470,封灵档案,Free to Play,"{'Sexual Content': 502, 'Free to Play': 342, '..."


In [511]:
def get_tags(tag_dict):
    tags = ast.literal_eval(tag_dict)
    tag_list = []
    for tag in tags:
        tag_list.append(tag)
        
    if len(tag_list) > 10:
        return tag_list[:10]
    else:
        return tag_list

In [512]:
game_data['tags'] = game_data['tags'].apply(get_tags)

In [513]:
game_data.tail(50)

,name,genre,tags
appid,,,
1220010,Home Behind 2,"Adventure, Indie, RPG, Simulation, Strategy, E...","[Strategy, RPG, Adventure, Indie, Simulation, ..."
1222140,Detroit: Become Human,"Action, Adventure","[Choices Matter, Story Rich, Multiple Endings,..."
1222670,The Sims 4,"Casual, Simulation","[Life Sim, Character Customization, Simulation..."
1222680,Need for Speed Heat,"Action, Adventure, Racing, Sports","[Racing, Open World, Multiplayer, Driving, Act..."
1222700,A Way Out,"Action, Adventure, Indie","[Co-op, Online Co-Op, Story Rich, Split Screen..."
1225330,NBA 2K21,"Simulation, Sports","[Basketball, Sports, Simulation, Character Cus..."
1226470,Shadow Arena,"Action, Free to Play, Massively Multiplayer, R...","[Action, Free to Play, Battle Royale, Massivel..."
1229490,ULTRAKILL,"Action, Indie, Early Access","[Early Access, FPS, Arena Shooter, Spectacle f..."
1237950,STAR WARS Battlefront II,"Action, Adventure","[Multiplayer, FPS, Singleplayer, Shooter, Thir..."


In [514]:
len(game_data)

1000

In [515]:
library_df = pd.read_csv('../data/library_data.csv')
library_df.head(50)

,steamid,library
0,76561198219067393,"[{'appid': 220, 'name': 'Half-Life 2', 'hours'..."
1,76561198148157441,"[{'appid': 17390, 'name': 'Spore', 'hours': 26..."
2,76561198993539076,hidden
3,76561198247182340,hidden
4,76561198278705159,hidden
5,76561198306000904,hidden
6,76561199041871881,hidden
7,76561198398210058,hidden
8,76561198313209867,hidden
9,76561198170079242,"[{'appid': 3830, 'name': 'Psychonauts', 'hours..."


In [516]:
len(library_df)

56978

In [517]:
library_df.set_index('steamid', inplace=True)
library_df

,library
steamid,
76561198219067393,"[{'appid': 220, 'name': 'Half-Life 2', 'hours'..."
76561198148157441,"[{'appid': 17390, 'name': 'Spore', 'hours': 26..."
76561198993539076,hidden
76561198247182340,hidden
76561198278705159,hidden
...,...
76561197990543347,hidden
76561199206760437,hidden
76561198324908021,hidden


In [518]:
library_df['library'].value_counts()

hidden                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [519]:
hidden_libraries = library_df[library_df['library'] == 'hidden'].index
library_df = library_df.drop(hidden_libraries)

In [520]:
len(library_df)

6600

In [521]:
library_df.drop_duplicates(inplace=True)

C:\Users\chris\anaconda3\envs\learn-env\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [522]:
len(library_df)

6340

In [523]:
library_df

,library
steamid,
76561198148157441,"[{'appid': 17390, 'name': 'Spore', 'hours': 26..."
76561198170079242,"[{'appid': 3830, 'name': 'Psychonauts', 'hours..."
76561198088650778,"[{'appid': 4000, 'name': ""Garry's Mod"", 'hours..."
76561198886682654,"[{'appid': 4000, 'name': ""Garry's Mod"", 'hours..."
76561198311899167,"[{'appid': 10, 'name': 'Counter-Strike', 'hour..."
...,...
76561198208253879,"[{'appid': 70, 'name': 'Half-Life', 'hours': 3..."
76561199104131020,"[{'appid': 230410, 'name': 'Warframe', 'hours'..."
76561198012694491,"[{'appid': 240, 'name': 'Counter-Strike: Sourc..."


In [524]:
library_df['library']

steamid
76561198148157441    [{'appid': 17390, 'name': 'Spore', 'hours': 26...
76561198170079242    [{'appid': 3830, 'name': 'Psychonauts', 'hours...
76561198088650778    [{'appid': 4000, 'name': "Garry's Mod", 'hours...
76561198886682654    [{'appid': 4000, 'name': "Garry's Mod", 'hours...
76561198311899167    [{'appid': 10, 'name': 'Counter-Strike', 'hour...
                                           ...                        
76561198208253879    [{'appid': 70, 'name': 'Half-Life', 'hours': 3...
76561199104131020    [{'appid': 230410, 'name': 'Warframe', 'hours'...
76561198012694491    [{'appid': 240, 'name': 'Counter-Strike: Sourc...
76561198393589724    [{'appid': 2600, 'name': 'Vampire: The Masquer...
76561198008893422    [{'appid': 6880, 'name': 'Just Cause', 'hours'...
Name: library, Length: 6340, dtype: object

In [525]:
def get_tags(appid):
    return game_data.loc[appid]['tags']

In [526]:
get_tags(10)

['Action',
 'FPS',
 'Multiplayer',
 'Shooter',
 'Classic',
 'Team-Based',
 'First-Person',
 'Competitive',
 'Tactical',
 "1990's"]

In [527]:
def tag_hours(games_list):
    games_list = ast.literal_eval(games_list)
    tag_dict = {}
    for game_dict in games_list:
        if game_dict['appid'] in game_data.index and game_dict['hours'] != 0:            
            tags = get_tags(game_dict['appid'])
            for tag in tags:
                if tag in tag_dict.keys():
                    tag_dict[tag] += game_dict['hours']
                else:
                    tag_dict[tag] = game_dict['hours']
    return dict(sorted(tag_dict.items(), key=lambda item: item[1], reverse=True))
        

In [528]:
print(tag_hours(library_df.iloc[0]['library']))

{'Action': 115061, 'Multiplayer': 112293, 'Free to Play': 76816, 'Singleplayer': 68393, 'RPG': 62375, 'Adventure': 61227, 'Open World': 57530, 'Shooter': 51982, 'Co-op': 51835, 'Third Person': 47150, 'Pixel Graphics': 43583, 'Fantasy': 39130, 'Action Roguelike': 36701, 'Parkour': 35250, 'Looter Shooter': 33962, 'Third-Person Shooter': 32866, 'Difficult': 31310, 'Sci-fi': 31309, 'Ninja': 31003, 'PvP': 30370, 'Indie': 29962, 'Story Rich': 28889, 'Dungeon Crawler': 26977, 'Atmospheric': 26177, 'Rogue-like': 26136, 'Competitive': 25216, 'Massively Multiplayer': 24811, 'Team-Based': 24583, 'Replay Value': 23898, 'Dark': 23898, 'Great Soundtrack': 23898, 'Survival': 22631, 'Mature': 22247, 'Strategy': 20729, 'Open World Survival Craft': 20725, 'First-Person': 20064, 'Choices Matter': 19195, 'Nudity': 19195, 'Crafting': 18054, 'Exploration': 17782, 'Building': 16908, 'MOBA': 16477, 'Mythology': 16093, 'Sandbox': 15456, 'Stealth': 14189, 'Character Customization': 13977, 'MMORPG': 13736, 'FPS'

In [529]:
library_df

,library
steamid,
76561198148157441,"[{'appid': 17390, 'name': 'Spore', 'hours': 26..."
76561198170079242,"[{'appid': 3830, 'name': 'Psychonauts', 'hours..."
76561198088650778,"[{'appid': 4000, 'name': ""Garry's Mod"", 'hours..."
76561198886682654,"[{'appid': 4000, 'name': ""Garry's Mod"", 'hours..."
76561198311899167,"[{'appid': 10, 'name': 'Counter-Strike', 'hour..."
...,...
76561198208253879,"[{'appid': 70, 'name': 'Half-Life', 'hours': 3..."
76561199104131020,"[{'appid': 230410, 'name': 'Warframe', 'hours'..."
76561198012694491,"[{'appid': 240, 'name': 'Counter-Strike: Sourc..."


In [530]:
library_df['tag_hours'] = library_df['library'].apply(tag_hours)
library_df

<ipython-input-530-7648b36c0f2c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  library_df['tag_hours'] = library_df['library'].apply(tag_hours)


,library,tag_hours
steamid,,
76561198148157441,"[{'appid': 17390, 'name': 'Spore', 'hours': 26...","{'Action': 115061, 'Multiplayer': 112293, 'Fre..."
76561198170079242,"[{'appid': 3830, 'name': 'Psychonauts', 'hours...","{'Multiplayer': 281821, 'Action': 213414, 'Fir..."
76561198088650778,"[{'appid': 4000, 'name': ""Garry's Mod"", 'hours...",{}
76561198886682654,"[{'appid': 4000, 'name': ""Garry's Mod"", 'hours...","{'Multiplayer': 109690, 'Singleplayer': 95062,..."
76561198311899167,"[{'appid': 10, 'name': 'Counter-Strike', 'hour...","{'Sandbox': 76219, 'Multiplayer': 74026, 'Firs..."
...,...,...
76561198208253879,"[{'appid': 70, 'name': 'Half-Life', 'hours': 3...","{'Multiplayer': 209252, 'Pixel Graphics': 1578..."
76561199104131020,"[{'appid': 230410, 'name': 'Warframe', 'hours'...","{'Multiplayer': 18877, 'Action': 18558, 'Shoot..."
76561198012694491,"[{'appid': 240, 'name': 'Counter-Strike: Sourc...","{'Multiplayer': 375807, 'Free to Play': 310345..."


In [531]:
library_df['tag_hours'].value_counts()

TypeError: unhashable type: 'dict'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'dict'


{}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [532]:
no_hours = library_df[library_df['tag_hours'] == {}].index
library_df = library_df.drop(no_hours)

In [533]:
len(library_df)

5738

In [534]:
def normalize_hours(tag_dict):
    tag_labels = tag_dict.keys()
    tag_hours = tag_dict.values()
    sum_hours = sum(tag_hours)
    norm_hours = [float(i)/sum_hours for i in tag_hours]
    return dict(zip(tag_labels, norm_hours))

In [535]:
print(normalize_hours(library_df.iloc[0]['tag_hours']))

{'Action': 0.06109671527032911, 'Multiplayer': 0.0596269235262258, 'Free to Play': 0.04078884487537568, 'Singleplayer': 0.03631628134192836, 'RPG': 0.03312075868440895, 'Adventure': 0.032511177426377665, 'Open World': 0.030548092138100953, 'Shooter': 0.027602136720367872, 'Co-op': 0.027524080583668745, 'Third Person': 0.02503637309771354, 'Pixel Graphics': 0.02314231704597347, 'Fantasy': 0.020777800197529814, 'Action Roguelike': 0.019488015462548983, 'Parkour': 0.018717542983974596, 'Looter Shooter': 0.018033622548134617, 'Third-Person Shooter': 0.01745165298471799, 'Difficult': 0.01662542612278708, 'Sci-fi': 0.01662489512865988, 'Ninja': 0.016462410925735162, 'PvP': 0.016126291643214427, 'Indie': 0.015909646039314804, 'Story Rich': 0.01533988934082389, 'Dungeon Crawler': 0.01432462856960802, 'Atmospheric': 0.013899833267844058, 'Rogue-like': 0.013878062508628655, 'Competitive': 0.013389547911600097, 'Massively Multiplayer': 0.013174495290082092, 'Team-Based': 0.013053428629079362, 'Re

In [536]:
library_df['tag_hours'] = library_df['tag_hours'].apply(normalize_hours)
library_df

<ipython-input-536-3514ccae6526>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  library_df['tag_hours'] = library_df['tag_hours'].apply(normalize_hours)


,library,tag_hours
steamid,,
76561198148157441,"[{'appid': 17390, 'name': 'Spore', 'hours': 26...","{'Action': 0.06109671527032911, 'Multiplayer':..."
76561198170079242,"[{'appid': 3830, 'name': 'Psychonauts', 'hours...","{'Multiplayer': 0.06902065567186039, 'Action':..."
76561198886682654,"[{'appid': 4000, 'name': ""Garry's Mod"", 'hours...","{'Multiplayer': 0.05836965139977544, 'Singlepl..."
76561198311899167,"[{'appid': 10, 'name': 'Counter-Strike', 'hour...","{'Sandbox': 0.06608431018935979, 'Multiplayer'..."
76561199063236653,"[{'appid': 500, 'name': 'Left 4 Dead', 'hours'...","{'Multiplayer': 0.08737094891821529, 'Action':..."
...,...,...
76561198208253879,"[{'appid': 70, 'name': 'Half-Life', 'hours': 3...","{'Multiplayer': 0.08415388451422459, 'Pixel Gr..."
76561199104131020,"[{'appid': 230410, 'name': 'Warframe', 'hours'...","{'Multiplayer': 0.09915953143877712, 'Action':..."
76561198012694491,"[{'appid': 240, 'name': 'Counter-Strike: Sourc...","{'Multiplayer': 0.09924078778500166, 'Free to ..."


In [537]:
library_df = library_df[~library_df.index.duplicated(keep='first')]

In [538]:
def rate_game_library(steamid):
    try:
        library = ast.literal_eval(library_df.loc[steamid]['library'])
    except ValueError:
        library = ast.literal_eval(library_df.loc[steamid]['library'].iloc[0])
    tag_hours = library_df.loc[steamid]['tag_hours']
    scores = {}
    for game in library:
        score = 0
        appid = game['appid']
        if appid in game_data.index:
            game_tags = game_data.loc[appid]['tags']
            for tag in game_tags:
                if tag in tag_hours:
                    score += tag_hours[tag]
        if appid in scores:
            scores[appid] += score
        else:
            scores[appid] = score
    return scores
        
    

In [539]:
rate_game_library(76561199095677278)
# print(library_df.loc[76561199095677278]['tag_hours'])

{236390: 0.18366166246653792,
 244210: 0.4656622311999295,
 251570: 0.1759023739716222,
 107410: 0.38017866073092044,
 270880: 0.5180837607005228,
 284160: 0.5229023053313854,
 21000: 0,
 213330: 0,
 313690: 0,
 355840: 0.19895764897382845,
 225540: 0.3032260911346232,
 577670: 0,
 585420: 0,
 605740: 0,
 636480: 0.13262274345221167,
 645630: 0.5497141624420236,
 648800: 0.18903030956746847,
 704850: 0.2849658269677685,
 715670: 0,
 873840: 0,
 787860: 0.7945176062207667,
 730: 0.10546180170816132,
 675010: 0.5437199086104275,
 1118200: 0.2224414351692962,
 1167630: 0.3117306164873849,
 1238840: 0.1852864749316049,
 1238860: 0.18923426912856317,
 1238810: 0.27452564693423287,
 424840: 0.08434120081191594,
 1248130: 0}

In [540]:
def game_ratings(df):
    index = df.index.values
#     print(index)
    library_ratings = []
    for steamid in index:
        rated_library = rate_game_library(steamid)
        library_ratings.append(rated_library)
    return library_ratings

In [541]:
library_df['game_ratings'] = library_df.index.to_series().apply(rate_game_library)
library_df

<ipython-input-541-47cf3a0f8e1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  library_df['game_ratings'] = library_df.index.to_series().apply(rate_game_library)


,library,tag_hours,game_ratings
steamid,,,
76561198148157441,"[{'appid': 17390, 'name': 'Spore', 'hours': 26...","{'Action': 0.06109671527032911, 'Multiplayer':...","{17390: 0.055111349468474874, 17440: 0, 550: 0..."
76561198170079242,"[{'appid': 3830, 'name': 'Psychonauts', 'hours...","{'Multiplayer': 0.06902065567186039, 'Action':...","{3830: 0.14023839496074098, 4000: 0.2444579416..."
76561198886682654,"[{'appid': 4000, 'name': ""Garry's Mod"", 'hours...","{'Multiplayer': 0.05836965139977544, 'Singlepl...","{4000: 0.21460970716729724, 400: 0.11458948611..."
76561198311899167,"[{'appid': 10, 'name': 'Counter-Strike', 'hour...","{'Sandbox': 0.06608431018935979, 'Multiplayer'...","{10: 0.2035782409655268, 80: 0.234358743150447..."
76561199063236653,"[{'appid': 500, 'name': 'Left 4 Dead', 'hours'...","{'Multiplayer': 0.08737094891821529, 'Action':...","{500: 0.3969835712361972, 3590: 0.114772421222..."
...,...,...,...
76561198054375336,"[{'appid': 220, 'name': 'Half-Life 2', 'hours'...","{'Open World': 0.08426903711625006, 'Adventure...","{220: 0.258722714828182, 320: 0.15850648804130..."
76561199104131020,"[{'appid': 230410, 'name': 'Warframe', 'hours'...","{'Multiplayer': 0.09915953143877712, 'Action':...","{230410: 0.5403950202237747, 238960: 0.1565162..."
76561198012694491,"[{'appid': 240, 'name': 'Counter-Strike: Sourc...","{'Multiplayer': 0.09924078778500166, 'Free to ...","{240: 0.22484247997000123, 300: 0.174308786792..."


In [543]:
def user_rating_df(steamid):
    big_df = pd.DataFrame(columns=['steamid', 'appid', 'rating'])
    rating_dict = library_df.loc[steamid]['game_ratings']
    for app, rating in rating_dict.items():
        big_df = big_df.append({'steamid': str(steamid), 'appid': str(app), 'rating': rating}, ignore_index=True)
    return big_df

In [ ]:
user_rating_df(76561198148157441)

In [555]:
user_rec_df = pd.DataFrame(columns=['steamid', 'appid', 'rating'])

In [556]:
for num, steamid in enumerate(library_df.index):
    total = len(library_df)
    print(f"{num} out of {total}")
    user_df = user_rating_df(steamid)
    user_rec_df = pd.concat([user_rec_df, user_df], ignore_index = True, axis = 0)
user_rec_df

0 out of 5518
1 out of 5518
2 out of 5518
3 out of 5518
4 out of 5518
5 out of 5518
6 out of 5518
7 out of 5518
8 out of 5518
9 out of 5518
10 out of 5518
11 out of 5518
12 out of 5518
13 out of 5518
14 out of 5518
15 out of 5518
16 out of 5518
17 out of 5518
18 out of 5518
19 out of 5518
20 out of 5518
21 out of 5518
22 out of 5518
23 out of 5518
24 out of 5518
25 out of 5518
26 out of 5518
27 out of 5518
28 out of 5518
29 out of 5518
30 out of 5518
31 out of 5518
32 out of 5518
33 out of 5518
34 out of 5518
35 out of 5518
36 out of 5518
37 out of 5518
38 out of 5518
39 out of 5518
40 out of 5518
41 out of 5518
42 out of 5518
43 out of 5518
44 out of 5518
45 out of 5518
46 out of 5518
47 out of 5518
48 out of 5518
49 out of 5518
50 out of 5518
51 out of 5518
52 out of 5518
53 out of 5518
54 out of 5518
55 out of 5518
56 out of 5518
57 out of 5518
58 out of 5518
59 out of 5518
60 out of 5518
61 out of 5518
62 out of 5518
63 out of 5518
64 out of 5518
65 out of 5518
66 out of 5518
67 ou

519 out of 5518
520 out of 5518
521 out of 5518
522 out of 5518
523 out of 5518
524 out of 5518
525 out of 5518
526 out of 5518
527 out of 5518
528 out of 5518
529 out of 5518
530 out of 5518
531 out of 5518
532 out of 5518
533 out of 5518
534 out of 5518
535 out of 5518
536 out of 5518
537 out of 5518
538 out of 5518
539 out of 5518
540 out of 5518
541 out of 5518
542 out of 5518
543 out of 5518
544 out of 5518
545 out of 5518
546 out of 5518
547 out of 5518
548 out of 5518
549 out of 5518
550 out of 5518
551 out of 5518
552 out of 5518
553 out of 5518
554 out of 5518
555 out of 5518
556 out of 5518
557 out of 5518
558 out of 5518
559 out of 5518
560 out of 5518
561 out of 5518
562 out of 5518
563 out of 5518
564 out of 5518
565 out of 5518
566 out of 5518
567 out of 5518
568 out of 5518
569 out of 5518
570 out of 5518
571 out of 5518
572 out of 5518
573 out of 5518
574 out of 5518
575 out of 5518
576 out of 5518
577 out of 5518
578 out of 5518
579 out of 5518
580 out of 5518
581 out 

1032 out of 5518
1033 out of 5518
1034 out of 5518
1035 out of 5518
1036 out of 5518
1037 out of 5518
1038 out of 5518
1039 out of 5518
1040 out of 5518
1041 out of 5518
1042 out of 5518
1043 out of 5518
1044 out of 5518
1045 out of 5518
1046 out of 5518
1047 out of 5518
1048 out of 5518
1049 out of 5518
1050 out of 5518
1051 out of 5518
1052 out of 5518
1053 out of 5518
1054 out of 5518
1055 out of 5518
1056 out of 5518
1057 out of 5518
1058 out of 5518
1059 out of 5518
1060 out of 5518
1061 out of 5518
1062 out of 5518
1063 out of 5518
1064 out of 5518
1065 out of 5518
1066 out of 5518
1067 out of 5518
1068 out of 5518
1069 out of 5518
1070 out of 5518
1071 out of 5518
1072 out of 5518
1073 out of 5518
1074 out of 5518
1075 out of 5518
1076 out of 5518
1077 out of 5518
1078 out of 5518
1079 out of 5518
1080 out of 5518
1081 out of 5518
1082 out of 5518
1083 out of 5518
1084 out of 5518
1085 out of 5518
1086 out of 5518
1087 out of 5518
1088 out of 5518
1089 out of 5518
1090 out of 55

1514 out of 5518
1515 out of 5518
1516 out of 5518
1517 out of 5518
1518 out of 5518
1519 out of 5518
1520 out of 5518
1521 out of 5518
1522 out of 5518
1523 out of 5518
1524 out of 5518
1525 out of 5518
1526 out of 5518
1527 out of 5518
1528 out of 5518
1529 out of 5518
1530 out of 5518
1531 out of 5518
1532 out of 5518
1533 out of 5518
1534 out of 5518
1535 out of 5518
1536 out of 5518
1537 out of 5518
1538 out of 5518
1539 out of 5518
1540 out of 5518
1541 out of 5518
1542 out of 5518
1543 out of 5518
1544 out of 5518
1545 out of 5518
1546 out of 5518
1547 out of 5518
1548 out of 5518
1549 out of 5518
1550 out of 5518
1551 out of 5518
1552 out of 5518
1553 out of 5518
1554 out of 5518
1555 out of 5518
1556 out of 5518
1557 out of 5518
1558 out of 5518
1559 out of 5518
1560 out of 5518
1561 out of 5518
1562 out of 5518
1563 out of 5518
1564 out of 5518
1565 out of 5518
1566 out of 5518
1567 out of 5518
1568 out of 5518
1569 out of 5518
1570 out of 5518
1571 out of 5518
1572 out of 55

1997 out of 5518
1998 out of 5518
1999 out of 5518
2000 out of 5518
2001 out of 5518
2002 out of 5518
2003 out of 5518
2004 out of 5518
2005 out of 5518
2006 out of 5518
2007 out of 5518
2008 out of 5518
2009 out of 5518
2010 out of 5518
2011 out of 5518
2012 out of 5518
2013 out of 5518
2014 out of 5518
2015 out of 5518
2016 out of 5518
2017 out of 5518
2018 out of 5518
2019 out of 5518
2020 out of 5518
2021 out of 5518
2022 out of 5518
2023 out of 5518
2024 out of 5518
2025 out of 5518
2026 out of 5518
2027 out of 5518
2028 out of 5518
2029 out of 5518
2030 out of 5518
2031 out of 5518
2032 out of 5518
2033 out of 5518
2034 out of 5518
2035 out of 5518
2036 out of 5518
2037 out of 5518
2038 out of 5518
2039 out of 5518
2040 out of 5518
2041 out of 5518
2042 out of 5518
2043 out of 5518
2044 out of 5518
2045 out of 5518
2046 out of 5518
2047 out of 5518
2048 out of 5518
2049 out of 5518
2050 out of 5518
2051 out of 5518
2052 out of 5518
2053 out of 5518
2054 out of 5518
2055 out of 55

...

3928 out of 5518
3929 out of 5518
3930 out of 5518
3931 out of 5518
3932 out of 5518
3933 out of 5518
3934 out of 5518
3935 out of 5518
3936 out of 5518
3937 out of 5518
3938 out of 5518
3939 out of 5518
3940 out of 5518
3941 out of 5518
3942 out of 5518
3943 out of 5518
3944 out of 5518
3945 out of 5518
3946 out of 5518
3947 out of 5518
3948 out of 5518
3949 out of 5518
3950 out of 5518
3951 out of 5518
3952 out of 5518
3953 out of 5518
3954 out of 5518
3955 out of 5518
3956 out of 5518
3957 out of 5518
3958 out of 5518
3959 out of 5518
3960 out of 5518
3961 out of 5518
3962 out of 5518
3963 out of 5518
3964 out of 5518
3965 out of 5518
3966 out of 5518
3967 out of 5518
3968 out of 5518
3969 out of 5518
3970 out of 5518
3971 out of 5518
3972 out of 5518
3973 out of 5518
3974 out of 5518
3975 out of 5518
3976 out of 5518
3977 out of 5518
3978 out of 5518
3979 out of 5518
3980 out of 5518
3981 out of 5518
3982 out of 5518
3983 out of 5518
3984 out of 5518
3985 out of 5518
3986 out of 55

4411 out of 5518
4412 out of 5518
4413 out of 5518
4414 out of 5518
4415 out of 5518
4416 out of 5518
4417 out of 5518
4418 out of 5518
4419 out of 5518
4420 out of 5518
4421 out of 5518
4422 out of 5518
4423 out of 5518
4424 out of 5518
4425 out of 5518
4426 out of 5518
4427 out of 5518
4428 out of 5518
4429 out of 5518
4430 out of 5518
4431 out of 5518
4432 out of 5518
4433 out of 5518
4434 out of 5518
4435 out of 5518
4436 out of 5518
4437 out of 5518
4438 out of 5518
4439 out of 5518
4440 out of 5518
4441 out of 5518
4442 out of 5518
4443 out of 5518
4444 out of 5518
4445 out of 5518
4446 out of 5518
4447 out of 5518
4448 out of 5518
4449 out of 5518
4450 out of 5518
4451 out of 5518
4452 out of 5518
4453 out of 5518
4454 out of 5518
4455 out of 5518
4456 out of 5518
4457 out of 5518
4458 out of 5518
4459 out of 5518
4460 out of 5518
4461 out of 5518
4462 out of 5518
4463 out of 5518
4464 out of 5518
4465 out of 5518
4466 out of 5518
4467 out of 5518
4468 out of 5518
4469 out of 55

4893 out of 5518
4894 out of 5518
4895 out of 5518
4896 out of 5518
4897 out of 5518
4898 out of 5518
4899 out of 5518
4900 out of 5518
4901 out of 5518
4902 out of 5518
4903 out of 5518
4904 out of 5518
4905 out of 5518
4906 out of 5518
4907 out of 5518
4908 out of 5518
4909 out of 5518
4910 out of 5518
4911 out of 5518
4912 out of 5518
4913 out of 5518
4914 out of 5518
4915 out of 5518
4916 out of 5518
4917 out of 5518
4918 out of 5518
4919 out of 5518
4920 out of 5518
4921 out of 5518
4922 out of 5518
4923 out of 5518
4924 out of 5518
4925 out of 5518
4926 out of 5518
4927 out of 5518
4928 out of 5518
4929 out of 5518
4930 out of 5518
4931 out of 5518
4932 out of 5518
4933 out of 5518
4934 out of 5518
4935 out of 5518
4936 out of 5518
4937 out of 5518
4938 out of 5518
4939 out of 5518
4940 out of 5518
4941 out of 5518
4942 out of 5518
4943 out of 5518
4944 out of 5518
4945 out of 5518
4946 out of 5518
4947 out of 5518
4948 out of 5518
4949 out of 5518
4950 out of 5518
4951 out of 55

5375 out of 5518
5376 out of 5518
5377 out of 5518
5378 out of 5518
5379 out of 5518
5380 out of 5518
5381 out of 5518
5382 out of 5518
5383 out of 5518
5384 out of 5518
5385 out of 5518
5386 out of 5518
5387 out of 5518
5388 out of 5518
5389 out of 5518
5390 out of 5518
5391 out of 5518
5392 out of 5518
5393 out of 5518
5394 out of 5518
5395 out of 5518
5396 out of 5518
5397 out of 5518
5398 out of 5518
5399 out of 5518
5400 out of 5518
5401 out of 5518
5402 out of 5518
5403 out of 5518
5404 out of 5518
5405 out of 5518
5406 out of 5518
5407 out of 5518
5408 out of 5518
5409 out of 5518
5410 out of 5518
5411 out of 5518
5412 out of 5518
5413 out of 5518
5414 out of 5518
5415 out of 5518
5416 out of 5518
5417 out of 5518
5418 out of 5518
5419 out of 5518
5420 out of 5518
5421 out of 5518
5422 out of 5518
5423 out of 5518
5424 out of 5518
5425 out of 5518
5426 out of 5518
5427 out of 5518
5428 out of 5518
5429 out of 5518
5430 out of 5518
5431 out of 5518
5432 out of 5518
5433 out of 55

,steamid,appid,rating
0,76561198148157441,17390,0.055111
1,76561198148157441,17440,0.0
2,76561198148157441,550,0.215663
3,76561198148157441,47870,0.083723
4,76561198148157441,65600,0.0
...,...,...,...
901104,76561198008893422,1546540,0.0
901105,76561198008893422,582660,0.228932
901106,76561198008893422,1551360,0.0
901107,76561198008893422,46500,0.0


In [552]:
user_rec_df

,steamid,appid,rating
0,76561198148157441,17390,0.055111
1,76561198148157441,17440,0.0
2,76561198148157441,550,0.215663
3,76561198148157441,47870,0.083723
4,76561198148157441,65600,0.0
...,...,...,...
708356,76561198370877979,494840,0.0
708357,76561198370877979,444200,0.084945
708358,76561198370877979,622650,0.110705
708359,76561198370877979,848450,0.376855


In [557]:
with open('big_rating_df.pickle', 'wb') as handle:
    pickle.dump(user_rec_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [558]:
with open('big_rating_df.pickle', 'rb') as handle:
    big_rating_df = pickle.load(handle)
big_rating_df

,steamid,appid,rating
0,76561198148157441,17390,0.055111
1,76561198148157441,17440,0.0
2,76561198148157441,550,0.215663
3,76561198148157441,47870,0.083723
4,76561198148157441,65600,0.0
...,...,...,...
901104,76561198008893422,1546540,0.0
901105,76561198008893422,582660,0.228932
901106,76561198008893422,1551360,0.0
901107,76561198008893422,46500,0.0


In [560]:
rating_matrix = big_rating_df.pivot_table(index='steamid', columns='appid', values='rating')
# replace NaN values with 0
rating_matrix = rating_matrix.fillna(0)
# display the top few rows
rating_matrix.head(50)

appid,10,100,10000,1000010,1000030,1000360,1000410,1000760,1001040,1001140,...,9990,99900,999020,99910,99920,999220,999660,999730,999820,999860
steamid,,,,,,,,,,,,,,,,,,,,,
76561197960328475,0.294394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197960428438,0.232919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197960447880,0.181719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.269084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197960495151,0.162066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197960520366,0.360615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [565]:
# display the top few rows
rating_matrix.describe()

appid,10,100,10000,1000010,1000030,1000360,1000410,1000760,1001040,1001140,...,9990,99900,999020,99910,99920,999220,999660,999730,999820,999860
count,5518.000000,5518.0,5518.0,5518.0,5518.0,5518.0,5518.0,5518.0,5518.0,5518.0,...,5518.0,5518.000000,5518.0,5518.0,5518.0,5518.0,5518.0,5518.0,5518.0,5518.0
mean,0.045510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.020089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.126262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.071068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.997766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.489307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [566]:
rating_matrix['20']

steamid
76561197960328475    0.304646
76561197960428438    0.219435
76561197960447880    0.147018
76561197960495151    0.153904
76561197960520366    0.290524
                       ...   
76561199219951385    0.000000
76561199221236273    0.000000
76561199221692308    0.000000
76561199222020759    0.000000
76561199222092780    0.000000
Name: 20, Length: 5518, dtype: float64